In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os

In [ ]:
train=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
train

In [ ]:
holiday=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
oil=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')

In [ ]:
train=pd.merge(train, holiday, on='date', how='left')
train=pd.merge(train, oil, on='date', how='left')

In [ ]:
stores=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')

In [ ]:
train=pd.merge(train, stores, on='store_nbr', how='left')

In [ ]:
transactions=pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')

In [ ]:
train=pd.merge(train, transactions, on=["date", "store_nbr"] , how='left')

In [ ]:
train.isnull().sum()

In [ ]:
train['date'] = pd.to_datetime(train['date'])
train['day'] = train['date'].dt.day
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year

In [ ]:
train['family'].unique()

In [ ]:
train.head()

In [ ]:
train['type_x'].unique()

In [ ]:
train['locale'].unique()

In [ ]:
train['description'].unique()

In [ ]:
train['city'].unique()